# Short cadence observations of HAT-P-7 b

HAT-P-7 b ([Pál et al. 2008](https://ui.adsabs.harvard.edu/abs/2008ApJ...680.1450P/abstract)) is a hot Jupiter orbiting a bright F6V host in the Kepler field. Let's take a look at short cadence (one-minute) Kepler observations from the first three Kepler quarters to see the secondary eclipse of the planet, when the planet's flux disappears as the planet is eclipsed by the host star.

First we download the observations with `lightkurve`:

In [ ]:
import astropy.units as u
from astropy.time import Time
from lightkurve import search_lightcurve

from lcviz import LCviz

# Download three quarters of short cadence observations from Kepler:
lc = search_lightcurve(
    target="HAT-P-7", 
    mission="Kepler", 
    quarter=[1, 2, 3], 
    cadence="short", 
    author="Kepler"
).download_all().stitch(
    # stitch together the quarters after normalizing each one:
    lambda x: x.normalize()
)

Now we'll start `LCviz`:

In [ ]:
lcviz = LCviz()
lcviz.show()

Now load the data. This takes a moment because there are nearly 300,000 observations in the light curve:

In [ ]:
lcviz.load_data(lc)

Now let's phase-fold the observations with the ephemeris of the planet using the `Ephemeris` plugin:

In [ ]:
# planet parameters from Morris et al. 2013:
# https://ui.adsabs.harvard.edu/abs/2013ApJ...764L..22M/abstract

time_coordinates = lcviz.app.data_collection[0].coords
reference_time = time_coordinates.reference_time

period = 2.204737  # days
epoch = Time(2454954.357463, format='jd')

# phase-fold the transit light curve in an ephemeris viewer:
eph = lcviz.plugins['Ephemeris']
eph.period = period
eph.t0 = (
    (epoch - reference_time).to_value(time_coordinates.unit) % eph.period
)

# offset the wrapping phase so the transit (at phase 0) displays at center
eph.wrap_at = 1

We can easily see the secondary eclipse if we bin the phased observations with the `Binning` plugin, and adjust the axes:

In [ ]:
binning = lcviz.plugins['Binning']
binning.n_bins = 150
binning.keep_active = True
binning.ephemeris = 'default'

viewer = lcviz.app.get_viewer('flux-vs-phase:default')
viewer.state.y_min = 0.9995
viewer.state.y_max = 1.0005